In [1]:
#!pip install Bani
import sys
from typing import List,Tuple,Dict

In [2]:
# appending paths as this is a jupyter notebook, if you want to use from the source code !!!
"""
sys.path.append('./')
sys.path.append('./core')
"""


"\nsys.path.append('./')\nsys.path.append('./core')\n"

In [3]:
from Bani.Bani import Bani
from Bani.core.FAQ import FAQ

In [4]:
"""
This cell is optional, we read the data from any faq here
In this case we use the orignal faq pickle files , you may use
csv or any other format !!!


THE goal here is to get a list of questions and corosponding answers
"""
import pickle
def save_dict(obj, path):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_dict(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
    
def extractQA(orignalDct):
    """
    Just returns the questions and answers in a list , 
    NOthing fancy
    """
    q2L = orignalDct["question_to_label"]
    a2L = orignalDct["answer_to_label"]
    

    l2A = dict()
    for a,l in a2L.items():
        l2A[l] = a


    questions = []
    answers = []

    for q,l in q2L.items():
        questions.append(q)
        answers.append(l2A[l])
    return questions , answers
    

babyBonusOrignal = load_dict("./tutorialFAQs/baby_bonus_orignal.pkl") 
comcareOrignal =  load_dict("./tutorialFAQs/comcare_orignal.pkl")


b_questions , b_answers = extractQA(babyBonusOrignal)
c_questions , c_answers = extractQA(comcareOrignal)



    


In [5]:
"""
NOw the real work starts , We have question answer pairs from 
baby bonus and comcare , We need to create a new faq for both of them

NOTE - This is the starting point , we need a LIst of questions and a list of answers
THE LENGTH OF BOTH LIST MUST BE SAME
if two questions answer the same questions then just copy the answer 
"""


# first we convert the questions and answers to FAQs
babyBonusFAQ = FAQ(name = "babyBonus",questions = b_questions, answers = b_answers)
comcareFAQ = FAQ(name = "comcare", questions = c_questions , answers = c_answers)

In [6]:
"""
Until this point we only have the orignal questions and answers in the faq
We now need to do two things
1) generate questions using pipelined
2) assign vectors to the questions using a model  


you can use your own generation pipeline , or use the default pipeline provided !!!
if you choose to implement your own pipeline make sure that your class implements at least one of 
batch_generate or exact_batch_generate methods


"""
from Bani.core.defaults import defaultGenerateManager


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['cls.predictions.decoder.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
"""
This is an example of creating your own GenerateManager

"""

from Bani.core.generation import GenerateManager

class MyProducer1:
    def __init__(self):
        pass
    
    def batch_generate(questions : List[str]) -> Dict[str, List[str]]:
        """
        Takes list of questions and returns a dict , with each question 
        mapped to the list of generated questions
        """
        
        resultDict = dict()
        for question in questions:
            resultDict[question] = ["generated1", "generated2", "and so on"]
        
        return resultDict
    
class MyProducer2:
    def __init__(self):
        pass
    
    def batch_generate(questions : List[str]) -> Dict[str, List[str]]:
        """
        Takes list of questions and returns a dict , with each question 
        mapped to the list of generated questions
        """
        
        resultDict = dict()
        for question in questions:
            resultDict[question] = ["generated1", "generated2", "and so on"]
        
        return resultDict
    
    
names = ["myProducer1_name", "myProducer2_name"]
toGenerate = [3,5] # At max generate 3 for first producer and 5 for second
producers = [MyProducer1(), MyProducer2()]


myGenerateManager = GenerateManager(producers = producers , names = names , nums = toGenerate)
    

    

    
    

    

In [8]:
"""

Now we will build the FAQ using the generationManager
This will generate paraphrases of the questions using the pipeline given

you can also leave the generator argument as none ,
In this case you wont be generating any questions, but then you cant train using a 
batch hard triple loss as well !!!

"""

babyBonusFAQ.buildFAQ(generator = defaultGenerateManager)

working with IdentityProducer pipeline


In [9]:
comcareFAQ.buildFAQ(generator  = defaultGenerateManager)

  0%|                                                                                          | 0/288 [00:00<?, ?it/s]

working with SymSub pipeline


  0%|                                                                                          | 0/288 [00:00<?, ?it/s]

working with FPM pipeline


 11%|████████▉                                                                       | 32/288 [00:00<00:00, 319.87it/s]

working with EDA pipeline


  0%|                                                                                          | 0/288 [00:00<?, ?it/s]

working with nlpAug pipeline


100%|████████████████████████████████████████████████████████████████████████████████| 288/288 [04:32<00:00,  1.06it/s]


In [10]:
"""
Now we can save the FAQ at any time we want !! , just pass the path to the root dir 
"""
babyBonusFAQ.save("./faqStore")
comcareFAQ.save("./faqStore")

In [6]:
"""
The saved FAQ can be easily loaded 
"""
babyBonusFAQ = FAQ("babyBonus")
babyBonusFAQ.load("./faqStore")

In [7]:
comcareFAQ = FAQ("comcare")
comcareFAQ.load("./faqStore")

In [8]:
"""
Lets now take a look at the bot part, for assignVectors refer to the readme
"""
bot = Bani(modelPath = None , FAQs =[babyBonusFAQ], assignVectors = False) 

C:\Users\rjkin\Desktop\BaniPackage\Bani\Bani.py:131: UserWarning: Vectors already assigned to babyBonus FAQ , if you want to reassign using the current model please clear the vectors using resetAssigned vectors
  warnings.warn("Vectors already assigned to {} FAQ , if you want to reassign using the current model please clear the vectors using resetAssigned vectors".format(faq.name))


In [8]:
"""
We can also save the FAQ again to preserve the assigned vectors ,
The next time you load these FAQs with the bot the vectors will not be reassigned ,
You need to clear the assigned vectors if you want to reassign
"""
bot.saveFAQs("./faqStore")


In [15]:
"""
out is a list of FAQ output , each output for each FAQ
compare the score field in each to get the highest scored FAQ, 
you can also set a margin of error and ask the user to decide between the faq

"""
out = bot.findClosest("what is comcare scheme?", K = 5)


In [16]:
print(out[0])

faqName - babyBonus

answer - The Child Development Account (CDA) trustee, can be either a parent or a third party nominated by the child’s parents as declared to MSF. Both parents must agree to the nomination. The CDA trustee’s responsibility is to ensure the proper usage of CDA funds for the benefit of the child. The CDA trustee must not be bankrupt, and must be above 18 years of age.

question - Who can be the Child Development Account (CDA) Trustee?

maxScore - 0.5368013978004456

score - 1.5918986797332764


['What is a Child Development Credit or CDA top-up?', 'What is the Child Development Co-Savings Scheme?', 'How can I identify an Approved Institution for the use of Child Development Account (CDA) funds?', 'How do Approved institutions collect fees from the Child Development Account (CDA)?', 'What is the Child Development Account (CDA)?']


In [18]:
"""
Finding answer from a specific FAQ
"""
out = bot.findClosestFromFAQ(0,"eligliblity for baby bonus for my child?", K = 3, topSimilar= 5)

In [19]:
print(out)

faqName - babyBonus

answer - You can check your child’s eligibility and Baby Bonus Benefits on Baby Bonus Online using “Check Eligibility”.    Alternatively, you can let us know which date your child is born to help us answer your question. Born on or after 24 March 2016 Please select your child birth order. You may wish to know that the birth order of the child is tied to his or her birth mother. A stillbirth is counted as a birth order. Non-citizen children and stepchildren are not considered in the counting of birth orders. First and Second Goes to MQA BBB - after 24 Mar 2016 - First and Second Third and Fourth Goes to MQA BBB - after 24 Mar 2016 – Third and Fourth Fifth and beyond Goes to MQA BBB - after 24 Mar 2016 – Fifth and Beyond Born on 1 January 2015 to 23 March 2016 Please select your child birth order. You may wish to know that the birth order of the child is tied to his or her birth mother. A stillbirth is counted as a birth order. Non-citizen children and stepchildren a

In [8]:
"""
Training
"""

'\nTraining\n'

In [12]:
"""
Look at the documentation for train , and set all the arguments accordingly,
if you have not run the buildFAQ , then you cannot train on most of the losses !!! 
losses available are batchHardTriplet and contrastiveLoss
"""
bot.train("./checkpoints/dummy", lossName = "softmaxLayerLoss", batchSize = 8, )

tensor(5.8670, grad_fn=<NllLossBackward>)
tensor(5.4983, grad_fn=<NllLossBackward>)
tensor(5.6927, grad_fn=<NllLossBackward>)
tensor(5.5118, grad_fn=<NllLossBackward>)
tensor(5.8105, grad_fn=<NllLossBackward>)
tensor(5.6536, grad_fn=<NllLossBackward>)
tensor(5.5936, grad_fn=<NllLossBackward>)
tensor(5.8394, grad_fn=<NllLossBackward>)
tensor(5.5624, grad_fn=<NllLossBackward>)
tensor(5.5640, grad_fn=<NllLossBackward>)
tensor(5.8930, grad_fn=<NllLossBackward>)
tensor(5.9448, grad_fn=<NllLossBackward>)
tensor(5.8004, grad_fn=<NllLossBackward>)
tensor(5.6623, grad_fn=<NllLossBackward>)
tensor(5.9043, grad_fn=<NllLossBackward>)
tensor(5.9048, grad_fn=<NllLossBackward>)
tensor(5.8339, grad_fn=<NllLossBackward>)
tensor(5.7747, grad_fn=<NllLossBackward>)
tensor(5.8096, grad_fn=<NllLossBackward>)
tensor(5.6965, grad_fn=<NllLossBackward>)
tensor(5.7411, grad_fn=<NllLossBackward>)
tensor(5.7340, grad_fn=<NllLossBackward>)
tensor(5.7139, grad_fn=<NllLossBackward>)
tensor(5.9498, grad_fn=<NllLossBac

KeyboardInterrupt: 

In [ ]:
"""
Testing
"""

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("./tutorialFAQs/babybonusTest.csv")

In [ ]:
df.columns

In [ ]:
testData = []

for i in range(len(df)):
    orignal = df.loc[i,"original"]
    reframed = df.loc[i,"reframed"]
    
    testData.append((orignal,reframed))

In [ ]:
"""
testing requires a list of (questions , reframed_question) to be passed and returns the accuracy score 
you can test one faq at a time
"""
bot.test(faqId = 0, testData = testData, K = 3)